In [69]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 
import random
import plotly.express as px
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt

https://docs.scipy.org/doc/scipy/reference/stats.contingency.html

https://docs.scipy.org/doc/scipy/reference/stats.mstats.html

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.contingency.relative_risk.html#scipy.stats.contingency.relative_risk

Anteilswerttest

sachen to do: schnellere methode für den diff abgleich finden
              
              schauen ob andere test dafür geiegnet sind

              geomap rumprobieren

In [2]:
df = pd.read_parquet("Shopping Carts.parquet")
df.reset_index(inplace=True)

In [3]:
df.groupby("order_id").agg(order_size=("add_to_cart_order","max")).reset_index()
df = pd.merge(df,df.groupby("order_id").agg(order_size=("add_to_cart_order","max")).reset_index())

In [4]:
df

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county,order_size
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo,8
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo,8
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo,8
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo,8
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132664,2493330,606914,28849,6,1,193225,18,5,13,4.0,1,No Salt Added Black Beans,59,15,canned goods,canned meals beans,Calaveras,8
6132665,3597250,606915,27966,1,0,107102,1,3,10,NaN,0,Organic Raspberries,123,4,produce,packaged vegetables fruits,Amador,4
6132666,3597251,606915,27845,4,0,107102,1,3,10,NaN,0,Organic Whole Milk,84,16,dairy eggs,milk,Amador,4
6132667,3597252,606915,26940,3,0,107102,1,3,10,NaN,0,Organic Large Green Asparagus,83,4,produce,fresh vegetables,Amador,4


### regionale unterschiede


In [5]:
df_regional = df.groupby("county").agg(count = ("order_id","nunique")).reset_index()

In [6]:
df_regional = df_regional.sort_values("count")

In [7]:
exceptions = df_regional.head(3).county.unique()

In [23]:
df_regional.tail(10)

,county,count
42,Santa Cruz,19340
20,Mariposa,23234
25,Monterey,24142
54,Ventura,24684
49,Sutter,25420
13,Kern,31601
8,Fresno,32685
28,Orange,34496
9,Glenn,35541
3,Calaveras,43559


große diskrepanz in den ausleihzahlen besonder San Bernardino, Placer, Trinity als problemfall

dazu map

#### top produkte und aisle

In [8]:
df_products = df.groupby("product_name").agg(coun = ("product_name","count")).reset_index().sort_values("coun",ascending=False)

In [31]:
df_products.tail(50)

,product_name,coun
32797,Pork Shoulder,1
10315,Creamy Supreme Frosting Chocolate,1
4711,Body Powder Original Strength,1
41094,The Amazing Prune Light Prune Juice Cocktail,1
19176,Imperial Champagne,1
10285,Creamy Oat Bran Hot Cereal,1
19119,"Iced Tea, Lemon",1
19117,Iced Tea with Raspberry,1
19027,Ice Cream Cake Celebration,1
19047,Ice Cream Super Chunky Cookie Dough,1


In [9]:
df_aisle = df.groupby("aisle").agg(coun = ("aisle","count")).reset_index().sort_values("coun",ascending=False)

In [10]:
df_aisle

,aisle,coun
50,fresh fruits,679534
53,fresh vegetables,639485
98,packaged vegetables fruits,333355
133,yogurt,274700
93,packaged cheese,184845
...,...,...
44,eye ear care,1744
3,baby bath body care,1686
2,baby accessories,1463
8,beauty,1195


#### oder_size je county

In [76]:
df

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county,order_size
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo,8
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo,8
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo,8
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo,8
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132664,2493330,606914,28849,6,1,193225,18,5,13,4.0,1,No Salt Added Black Beans,59,15,canned goods,canned meals beans,Calaveras,8
6132665,3597250,606915,27966,1,0,107102,1,3,10,NaN,0,Organic Raspberries,123,4,produce,packaged vegetables fruits,Amador,4
6132666,3597251,606915,27845,4,0,107102,1,3,10,NaN,0,Organic Whole Milk,84,16,dairy eggs,milk,Amador,4
6132667,3597252,606915,26940,3,0,107102,1,3,10,NaN,0,Organic Large Green Asparagus,83,4,produce,fresh vegetables,Amador,4


In [79]:
df_order_size = df.groupby(["order_id","county"])[["order_size"]].max().reset_index()

In [83]:
df_order_size = df_order_size.groupby("county")[["order_size"]].median().reset_index()

In [86]:
df_order_size.sort_values("order_size")

,county,order_size
0,Alameda,3.0
10,Humboldt,3.0
34,San Bernardino,3.0
29,Placer,3.5
4,Colusa,5.0
21,Mendocino,5.0
26,Napa,5.0
2,Amador,5.0
48,Stanislaus,6.0
47,Sonoma,6.0


#### top aisle pro county


In [11]:
df_3 = df.groupby(["county","aisle"]).agg(anzahl=("aisle","count")).reset_index()

In [12]:
counties = df_3.county.unique().tolist()
for i in counties:
    df_3.loc[df_3["county"] == i,"rank"] = df_3.loc[df_3["county"] == i].anzahl.rank(method="min",ascending=False)

In [37]:
df_3

,county,aisle,anzahl,rank
0,Alameda,air fresheners candles,6,115.0
1,Alameda,asian foods,10,107.0
2,Alameda,baby accessories,1,126.0
3,Alameda,baby bath body care,1,126.0
4,Alameda,baby food formula,29,77.0
...,...,...,...,...
7336,Yuba,trash bags liners,187,97.0
7337,Yuba,vitamins supplements,314,80.0
7338,Yuba,water seltzer sparkling water,7977,3.0
7339,Yuba,white wines,375,71.0


#### top 10 aisled pro county

In [13]:
df_4 = df_3[df_3["rank"].isin(range(11))]
df_4 = df_4.sort_values(["county","rank"])

In [14]:
df_4

,county,aisle,anzahl,rank
49,Alameda,fresh fruits,2824,1.0
52,Alameda,fresh vegetables,982,2.0
132,Alameda,yogurt,933,3.0
130,Alameda,water seltzer sparkling water,821,4.0
97,Alameda,packaged vegetables fruits,610,5.0
...,...,...,...,...
7290,Yuba,milk,4686,6.0
7232,Yuba,chips pretzels,4492,7.0
7249,Yuba,energy granola bars,3931,8.0
7324,Yuba,soft drinks,3830,9.0


In [40]:
df_3

,county,aisle,anzahl,rank
0,Alameda,air fresheners candles,6,115.0
1,Alameda,asian foods,10,107.0
2,Alameda,baby accessories,1,126.0
3,Alameda,baby bath body care,1,126.0
4,Alameda,baby food formula,29,77.0
...,...,...,...,...
7336,Yuba,trash bags liners,187,97.0
7337,Yuba,vitamins supplements,314,80.0
7338,Yuba,water seltzer sparkling water,7977,3.0
7339,Yuba,white wines,375,71.0


#### frage wegen mengen bias (weil counties ja unterschiedliche ausleihzeilen haben dann den durchschnitt von je county nehmen oder den nochmal auf gesmatmenge projezieren)

In [15]:
df_5 = df_3.groupby("aisle").agg(rank_mean=("rank","mean")).reset_index()

In [16]:
df_6 = pd.merge(df_3,df_5)

In [17]:
df_8 = df_6.loc[(df_6["rank"] > df_6.rank_mean * 1.5)]

In [18]:
df_8[df_8.county.isin(exceptions)]

,county,aisle,anzahl,rank,rank_mean
1301,Trinity,cereal,1,75.0,22.854545
1358,Trinity,chips pretzels,9,26.0,9.719298
1690,Trinity,cookies cakes,1,75.0,39.666667
1730,San Bernardino,crackers,1,30.0,18.947368
1747,Trinity,crackers,8,30.0,18.947368
2230,San Bernardino,eggs,1,30.0,17.771930
2302,Trinity,energy granola bars,1,75.0,22.854545
2725,San Bernardino,fresh fruits,5,11.0,1.508772
2888,Placer,fresh vegetables,8,6.0,2.315789
2893,San Bernardino,fresh vegetables,3,19.0,2.315789


In [19]:
df_9 = df_6.loc[(df_6["rank"] < df_6.rank_mean * 0.5)]

In [20]:
df_9

,county,aisle,anzahl,rank,rank_mean
29,Placer,air fresheners candles,1,36.0,108.600000
104,Trinity,asian foods,10,25.0,52.290909
212,Alpine,baby food formula,2647,9.0,34.600000
219,Fresno,baby food formula,5425,12.0,34.600000
220,Glenn,baby food formula,6390,13.0,34.600000
...,...,...,...,...,...
7231,Trinity,white wines,7,32.0,97.693878
7277,Sierra,yogurt,1567,2.0,4.018182
7283,Tehama,yogurt,718,1.0,4.018182
7289,Yuba,yogurt,8243,2.0,4.018182


In [21]:
df_9[df_9.county.isin(exceptions)]

,county,aisle,anzahl,rank,rank_mean
29,Placer,air fresheners candles,1,36.0,108.600000
104,Trinity,asian foods,10,25.0,52.290909
510,Placer,beers coolers,2,26.0,84.634615
515,San Bernardino,beers coolers,8,4.0,84.634615
567,San Bernardino,body lotions soap,4,13.0,85.836364
...,...,...,...,...,...
6619,Placer,specialty wines champagnes,1,36.0,112.854167
6723,Placer,spirits,3,20.0,95.979592
6728,San Bernardino,spirits,2,24.0,95.979592
6833,San Bernardino,tea,6,6.0,38.803571


In [173]:
df_2 = df.copy()
for i in df_9.head().values:
    
    df_11 = df_2[df_2.aisle == i[1]].groupby("county")[["aisle"]].count().reset_index()
    merged = df_2.merge(df_2[df_2.aisle == i[1]],indicator=True,how="outer")
    merged = merged.loc[merged["_merge"] == "left_only"]
    df_test = pd.merge(df_11,merged.groupby("county").agg(not_aisle=("aisle","count")).reset_index())
    df_test.loc[df_test.county != i[0],["aisle","not_aisle"]] = df_test.loc[df_test.county != i[0]].aisle.sum(),df_test.loc[df_test.county != i[0]].not_aisle.sum()
    ch2, p, dof, expected = chi2_contingency(df_test.loc[(df_test.county =="Alameda") | (df_test.county ==i[0]) ].set_index("county"))
    print(p)

0.3886016871794329
0.430366635739061
3.079459716602713e-160
3.360125579776243e-75
7.48389484632632e-154


In [184]:
df_2 = df.copy()
for i in df_9.head().values:
    
    df_11 = df_2[df_2.aisle == i[1]].groupby("county")[["aisle"]].count().reset_index()
    merged = df_2[~(df_2.index.isin(df_2[df_2.aisle == i[1]].index))]
    df_test = pd.merge(df_11,merged.groupby("county").agg(not_aisle=("aisle","count")).reset_index())
    df_test.loc[df_test.county != i[0],["aisle","not_aisle"]] = df_test.loc[df_test.county != i[0]].aisle.median(),df_test.loc[df_test.county != i[0]].not_aisle.median()
    ch2, p, dof, expected = chi2_contingency(df_test.loc[(df_test.county =="Alameda") | (df_test.county ==i[0]) ].set_index("county"))
    print(p)

0.3613479235501478
0.22669578631959336
1.379306155077659e-78
3.118630641189331e-38
1.2917867215753157e-50


In [171]:
df_9.head()

,county,aisle,anzahl,rank,rank_mean
29,Placer,air fresheners candles,1,36.0,108.600000
104,Trinity,asian foods,10,25.0,52.290909
212,Alpine,baby food formula,2647,9.0,34.600000
219,Fresno,baby food formula,5425,12.0,34.600000
220,Glenn,baby food formula,6390,13.0,34.600000


In [172]:
df_9.head().apply(lambda x: print(x),axis=1)

county                       Placer
aisle        air fresheners candles
anzahl                            1
rank                           36.0
rank_mean                     108.6
Name: 29, dtype: object
county           Trinity
aisle        asian foods
anzahl                10
rank                25.0
rank_mean      52.290909
Name: 104, dtype: object
county                  Alpine
aisle        baby food formula
anzahl                    2647
rank                       9.0
rank_mean                 34.6
Name: 212, dtype: object
county                  Fresno
aisle        baby food formula
anzahl                    5425
rank                      12.0
rank_mean                 34.6
Name: 219, dtype: object
county                   Glenn
aisle        baby food formula
anzahl                    6390
rank                      13.0
rank_mean                 34.6
Name: 220, dtype: object


29     None
104    None
212    None
219    None
220    None
dtype: object

In [176]:
df_2 = df.copy()

In [30]:
df_2

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county,order_size
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo,8
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo,8
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo,8
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo,8
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132664,2493330,606914,28849,6,1,193225,18,5,13,4.0,1,No Salt Added Black Beans,59,15,canned goods,canned meals beans,Calaveras,8
6132665,3597250,606915,27966,1,0,107102,1,3,10,NaN,0,Organic Raspberries,123,4,produce,packaged vegetables fruits,Amador,4
6132666,3597251,606915,27845,4,0,107102,1,3,10,NaN,0,Organic Whole Milk,84,16,dairy eggs,milk,Amador,4
6132667,3597252,606915,26940,3,0,107102,1,3,10,NaN,0,Organic Large Green Asparagus,83,4,produce,fresh vegetables,Amador,4


In [177]:
df_11 = df_2[df_2.aisle == "air fresheners candles"].groupby("county")[["aisle"]].count().reset_index()

In [178]:
df_11

,county,aisle
0,Alameda,6
1,Alpine,90
2,Amador,69
3,Calaveras,388
4,Colusa,19
5,Contra Costa,72
6,Del Norte,10
7,El Dorado,1
8,Fresno,143
9,Glenn,310


In [181]:
merged = df_2[~(df_2.index.isin(df_2[df_2.aisle == "air fresheners candles"].index))]

In [182]:
merged

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county,order_size
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo,8
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo,8
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo,8
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo,8
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132664,2493330,606914,28849,6,1,193225,18,5,13,4.0,1,No Salt Added Black Beans,59,15,canned goods,canned meals beans,Calaveras,8
6132665,3597250,606915,27966,1,0,107102,1,3,10,NaN,0,Organic Raspberries,123,4,produce,packaged vegetables fruits,Amador,4
6132666,3597251,606915,27845,4,0,107102,1,3,10,NaN,0,Organic Whole Milk,84,16,dairy eggs,milk,Amador,4
6132667,3597252,606915,26940,3,0,107102,1,3,10,NaN,0,Organic Large Green Asparagus,83,4,produce,fresh vegetables,Amador,4


In [146]:
merged = df_2.merge(df_2[df_2.aisle == "air fresheners candles"],indicator=True,how="outer")

In [147]:
merged = merged.loc[merged["_merge"] == "left_only"]

In [148]:
merged

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county,order_size,_merge
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo,8,left_only
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo,8,left_only
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo,8,left_only
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo,8,left_only
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo,8,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132664,2493330,606914,28849,6,1,193225,18,5,13,4.0,1,No Salt Added Black Beans,59,15,canned goods,canned meals beans,Calaveras,8,left_only
6132665,3597250,606915,27966,1,0,107102,1,3,10,NaN,0,Organic Raspberries,123,4,produce,packaged vegetables fruits,Amador,4,left_only
6132666,3597251,606915,27845,4,0,107102,1,3,10,NaN,0,Organic Whole Milk,84,16,dairy eggs,milk,Amador,4,left_only
6132667,3597252,606915,26940,3,0,107102,1,3,10,NaN,0,Organic Large Green Asparagus,83,4,produce,fresh vegetables,Amador,4,left_only


In [149]:
merged.groupby("county").agg(not_aisle=("aisle","count")).reset_index()

,county,not_aisle
0,Alameda,16910
1,Alpine,133193
2,Amador,121043
3,Calaveras,502091
4,Colusa,34924
5,Contra Costa,96649
6,Del Norte,19300
7,El Dorado,15422
8,Fresno,357798
9,Glenn,390104


In [150]:
df_test = pd.merge(df_11,merged.groupby("county").agg(not_aisle=("aisle","count")).reset_index())

In [153]:
df_test.loc[df_test.county != "Placer",["aisle","not_aisle"]] = df_test.loc[df_test.county != "Placer"].aisle.sum(),df_test.loc[df_test.county != "Placer"].not_aisle.sum()

In [160]:
df_test.loc[(df_test.county =="Alameda") | (df_test.county =="Placer") ]

,county,aisle,not_aisle
0,Alameda,3747,6127097
29,Placer,1,257


In [71]:
df_test = df_test.set_index("county")

KeyError: "None of ['county'] are in the columns"

In [72]:
df_test

,aisle,not_aisle
county,,
Alameda,6,16910
Alpine,90,133193
Amador,69,121043
Calaveras,388,502091
Colusa,19,34924
Contra Costa,72,96649
Del Norte,10,19300
El Dorado,1,15422
Fresno,143,357798


In [161]:
ch2, p, dof, expected = chi2_contingency(df_test.loc[(df_test.county =="Alameda") | (df_test.county =="Placer") ].set_index("county"))

In [162]:
p

0.3886016871794329

In [163]:
expected

array([[3.74784228e+03, 6.12709616e+06],
       [1.57717813e-01, 2.57842282e+02]])

In [89]:
df_test = df_test.reset_index()

In [94]:
df_test.loc[df_test.county != "Placer"].aisle.sum()

3747

In [95]:
df_test.loc[df_test.county != "Placer"]["all"] = 0

C:\Users\paulh\AppData\Local\Temp\ipykernel_16080\1065071676.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.loc[df_test.county != "Placer"]["all"] = 0


In [98]:
df_test

,county,aisle,not_aisle
0,Alameda,6,16910
1,Alpine,90,133193
2,Amador,69,121043
3,Calaveras,388,502091
4,Colusa,19,34924
5,Contra Costa,72,96649
6,Del Norte,10,19300
7,El Dorado,1,15422
8,Fresno,143,357798
9,Glenn,310,390104


In [97]:
df_test.loc[df_test.county != "Placer"].append(["all",df_test.loc[df_test.county != "Placer"].aisle.sum(),df_test.loc[df_test.county != "Placer"].not_aisle.sum()])

C:\Users\paulh\AppData\Local\Temp\ipykernel_16080\2367690333.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test.loc[df_test.county != "Placer"].append(["all",df_test.loc[df_test.county != "Placer"].aisle.sum(),df_test.loc[df_test.county != "Placer"].not_aisle.sum()])


,county,aisle,not_aisle,0
0,Alameda,6.0,16910.0,NaN
1,Alpine,90.0,133193.0,NaN
2,Amador,69.0,121043.0,NaN
3,Calaveras,388.0,502091.0,NaN
4,Colusa,19.0,34924.0,NaN
5,Contra Costa,72.0,96649.0,NaN
6,Del Norte,10.0,19300.0,NaN
7,El Dorado,1.0,15422.0,NaN
8,Fresno,143.0,357798.0,NaN
9,Glenn,310.0,390104.0,NaN


In [90]:
df_test["alle"] 

,county,aisle,not_aisle
0,Alameda,6,16910
1,Alpine,90,133193
2,Amador,69,121043
3,Calaveras,388,502091
4,Colusa,19,34924
5,Contra Costa,72,96649
6,Del Norte,10,19300
7,El Dorado,1,15422
8,Fresno,143,357798
9,Glenn,310,390104
